In [ ]:
from pprint import pprint
from tqdm import tqdm
from pdf2image import convert_from_path

from PIL import Image

import easyocr
reader = easyocr.Reader(['fa','en'])

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH

from collections.abc import Sequence

import os
import tempfile

def pdf_to_word(pdf_path, output_dir, lang='fas+eng', **kwargs):
    """ 
    A way to use google-tesseract ocr for extracting
    texts from pdf file.
    
    Args:
        pdf_path (str): PDF file path.
        output_dir (str): Output directory.
        lang (str): tesseract language support.
    """
    
    pdf_name = pdf_path.split('/')[-1].split('.')[0]
    pages = convert_from_path(pdf_path, **kwargs)
    texts = []
    
    print(f'PDF is preparing to convert into document [#{len(pages)} pages]')
    for i, page in tqdm(enumerate(pages), position=0):
        
        with tempfile.TemporaryDirectory() as img_dir:
            img_name = f'{pdf_name}-{i+1}.jpg'
            img_path = os.path.join(img_dir, img_name)

            page.save(img_path, 'JPEG')
            # text = pytesseract.image_to_string(Image.open(img_path), lang=lang)
            text = reader.readtext(img_path)
            # texts.append(text)
            texts += text;
    
    return texts
    # with open('readme.txt', 'w') as f:
        # f.write(str(texts))



In [ ]:
t = pdf_to_word('./samples/sss.pdf', './samples/')

In [ ]:
for x in t:
    print(x)

In [ ]:
t[1][0]

In [ ]:
t[1][0][2]

In [ ]:
t[1][0][1]

In [ ]:
mx_line = 0;
for i in range(0, len(t)):
    tmp = t[i][0][2][1] - t[i][0][1][1]; 
    if tmp > mx_line:
        mx_line = tmp
mx_line

In [8]:
words = []
for i in range(0,len(t)):
    tmp = []
    y = (t[i][0][2][1] + t[i][0][1][1]) / 2

    tmp.append([y, t[i][0][2][0]])
    tmp.append(t[i][1])
    words.append(tmp)

In [ ]:
words = sorted(words)
words

In [10]:
lines = []
line = []
pre = words[0][0][0]
for word in words:
    diff = abs(word[0][0] - pre)

    if diff <= 10:
        line.append(word)
    else:
        lines.append(line)
        line =[word]
        pre = word[0][0]

if line:
    lines.append(line)

In [ ]:
lines[0]

In [16]:
for line in lines:
    for word in line:
        word[0].reverse()

In [ ]:
lines

In [20]:
for i in range(0, len(lines)):
    lines[i].sort(reverse=True)

In [ ]:
lines